# Erster Agent

Es ist an der Zeit einen Agenten zu erstellen. Und zwar werden wir einen Agenten mit zwei-drei Tools erstellen. 
Als erstes Tool möchten wir dem Agenten Zugriff auf Wikipedia geben, damit der Agent mithilfe der Wikipedia-Artikel fundierte Antworten geben kann.
Als zweites Tool bekommt der die DuckDuckGo-Funktion.



In [ ]:
!pip install langchain langchain_openai python-dotenv duckduckgo-search wikipedia

In [21]:
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o")

In [ ]:
search = DuckDuckGoSearchResults()

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [search, wikipedia]

In [ ]:
system_template = "Du bist ein hilfreicher Assistent, der in der Lage ist die notwendigen Fakten im Internet zu suchen oder bei Wikipedia nachzuschlagen."

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_template)),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt_template)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
result = agent_executor.invoke({"input": "Guten Tag! Wann wurde Budeskanuzler Olfa Scholz geboren?"})

result

In [ ]:
result = agent_executor.invoke({"input": "Wann findet das Münchner Oktoberfest statt?"})

result